<a href="https://colab.research.google.com/github/rahiakela/transfer-learning-for-natural-language-processing/blob/main/3-shallow-transfer-learning-for-nlp/4_transfer_learning_using_domain_adaptation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Transfer Learning using Domain Adaptation

In this notebook, we will cover some prominent shallow transfer learning approaches and concepts. This allows us to explore some major themes in transfer learning, while doing so in the context of relatively simple models of the class of eventual interest, i.e., shallow neural networks.

Roughly speaking, categorization is based on whether transfer occurs between different languages, tasks or data domains. Each of these types of categorization is usually correspondingly referred to as cross-lingual learning, multi-task learning and domain adaptation.

<img src='https://github.com/rahiakela/img-repo/blob/master/transfer-learning-for-natural-language-processing/shallow-transfer-learning.png?raw=1' width='800'/>

The methods we will look at here will involve components that are neural networks in one way or another.these neural networks do not have many
layers. This is the reason why the label “shallow” is appropriate to describe this collection of methods.

A common form of semi-supervised learning that employs pretrained word embeddings such as word2vec that they produce a single vector per word, regardless of context.

We revisit the IMDB movie review sentiment classification. Recall that this example is concerned with classifying movie reviews from IMDB into positive or negative sentiments expressed. It is a prototypical sentiment analysis example that has been used widely in the literature to study many algorithms. We combine feature vectors generated by pretrained word embeddings for each review with some traditional machine learning classification methods, namely random forests and logistic regression.

We then demonstrate that using higher-level embeddings which vectorize bigger sections of text – such as at the sentence-level, paragraphlevel and document-level – can lead to improved performance. The general idea of vectorizing text and then applying a traditional machine learning classification method to the resulting vectors.

<img src='https://github.com/rahiakela/img-repo/blob/master/transfer-learning-for-natural-language-processing/semi-supervised-learning.png?raw=1' width='800'/>

**Multi-task learning**

Subsequently, we introduce the reader to multi-task learning. We demonstrate how one can train a single system simultaneously to perform multiple tasks, email spam classification and IMDB movie review sentiment analysis. 

There are several potential benefits to multi-task learning. By
training a single machine learning model for multiple tasks, a shared representation is learned on a larger and more varied collection of data from the combined data pool, which can lead to performance improvements. Moreover, it has been widely observed that this shared representation has a better ability to generalize to tasks beyond those that were trained on, and
this improvement can be achieved without any increase in model size.

Specifically, we focus on shallow neural multitask learning, where a single additional dense layer, as well as a classification layer, is trained
for each specific task in the setup. Different tasks also share a layer between them, a setup typically referred to as hard-parameter sharing.

**Domain adaptation**

Assume that we are given one source domain, which can be defined as a particular distribution of data for a specific task, and a classifier that has been trained to perform well on data in that domain for that task. The goal of domain adaptation is to modify, or adapt, data in a different target domain in such a way that the pretrained knowledge from the source domain can aid
learning in the target domain. We apply a simple autoencoding approach to “project” samples in the target domain into the source domain feature space.

An autoencoder is a system that learns to reconstruct inputs with very high accuracy, typically by encoding them into an efficient latent representation and learning to decode the said representation efficiently. They have traditionally been heavily used in model reduction applications, since the latent representation is often of smaller dimension than the original space
from which the encoding happens, and the said dimension value can also be picked to strike the right balance of computational efficiency and accuracy.

In the extreme scenario, improvements can be obtained with no labelled data in the target domain being used for training. This is typically referred to as zero-shot domain adaptation, where learning happens with no labeled
data in the target domain.


## Setup

In [1]:
import tensorflow as tf

print(tf.__version__)

2.4.1


In [ ]:
# install sent2vec
!pip install git+https://github.com/epfml/sent2vec

In [21]:
import numpy as np  # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import re
import os
import time
import email

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, concatenate

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import sent2vec

import matplotlib.pyplot as plt
from IPython.display import HTML

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Download IMDB Movie Review Dataset

In [4]:
%%shell

wget -q "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
tar xzf aclImdb_v1.tar.gz

rm -rf aclImdb_v1.tar.gz
rm -rf aclImdb/train/unsup

Download Book Reviews dataset.

In [8]:
%%shell

wget -q "https://github.com/rahiakela/transfer-learning-for-natural-language-processing/raw/main/3-shallow-transfer-learning-for-nlp/datasets/books.review.zip"
unzip -o books.review.zip

rm -rf books.review.zip

Archive:  books.review.zip
  inflating: books.negative.review   
  inflating: books.positive.review   
  inflating: dvd.negative.review     
  inflating: dvd.positive.review     


Let's download sent2vec word Embedding from [Kaggle](https://www.kaggle.com/maxjeblick/sent2vec)

In [ ]:
from google.colab import files
files.upload() # upload kaggle.json file

In [13]:
%%shell

mkdir -p ~/.kaggle
mv kaggle.json ~/.kaggle/
ls ~/.kaggle
chmod 600 /root/.kaggle/kaggle.json

# download word embeddings from kaggle
kaggle datasets download -d maxjeblick/sent2vec/wiki_unigrams.bin
unzip -qq sent2vec.zip
rm -rf sent2vec.zip

kaggle.json
100% 4.42G/4.43G [01:27<00:00, 69.4MB/s]
100% 4.43G/4.43G [01:27<00:00, 54.3MB/s]


In [14]:
def extract_messages(df):
  messages = []
  for item in df["message"]:
    # Return a message object structure from a string
    e = email.message_from_string(item)
    # get message body
    message_body = e.get_payload()
    messages.append(message_body)
  print("Successfully retrieved message body from e-mails!")
  return messages

## Preprocessing IMDB Data

Before proceeding, we must decide how many samples to draw from each class. We must also decide the maximum number of tokens per email, and the maximum length of each token. This is done by setting the following overarching hyperparameters.

In [15]:
n_sample = 1000    # number of samples to generate in each class
maxtokens = 200    # the maximum number of tokens per document
maxtokenlen = 100  # the maximum length of each token

### Tokenization

Let’s proceed by defining a function to tokenize text by splitting them into 
words.

In [16]:
def tokenize(row):
  if row is None or row is "":
    tokens = ""
  else:
    tokens = str(row).split(" ")[:maxtokens]
  return tokens

### Remove punctuation and unnecessary characters

**In order to ensure that classification is done based on language content only, we have to remove punctuation marks and other non-word characters from the emails.** We do this by employing regular expressions with the Python regex library. We also normalize words by turning them into lower case.

In [17]:
def reg_expressions(row):
  tokens = []
  try:
    for token in row:
      token = token.lower()          # make all characters lower case
      token = re.sub(r"[\W\d]", "", token)
      token = token[:maxtokenlen]    # truncate all tokens to hyperparameter maxtokenlen
      tokens.append(token)
  except:
    token = ""
    tokens.append(token)
  return tokens

### Stop-word removal

Stop-words are also removed. Stop-words are words that are very common in text but offer no useful information that can be used to classify the text. Words such as is, and, the, are are examples of stop-words. The NLTK library contains a list of 127 English stop-words and can be used to filter our tokenized strings.

In [18]:
stop_words = stopwords.words("english")
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
# print(stopwords) # see default stopwords
# it may be beneficial to drop negation words from the removal list, as they can change the positive/negative meaning
# of a sentence
# stopwords.remove("no")
# stopwords.remove("nor")
# stopwords.remove("not")

In [19]:
def stop_word_removal(row):
  token = [token for token in row if token not in stop_words]
  token = filter(None, token)

  return token

### Load pre-trained sent2vec embedding

Quite naturally, just as in the case of the pretrained word embeddings, the next step is to obtain the pretrained sent2vec sentence embedding to be loaded by the particular implementation/framework installed.

We choose the smallest 600-dimensional embedding `wiki_unigrams.bin`, approximately 5 Gigabytes in size, which captures just the unigram information on Wikipedia.

Now let's load the pre-trained embedding.

In [22]:
# load sent2vec embedding
s2v_model = sent2vec.Sent2vecModel()

start = time.time()
s2v_model.load_model("wiki_unigrams.bin")
end = time.time()

print("Loading the sent2vec embedding took %d seconds" % (end - start))

Loading the sent2vec embedding took 17 seconds


### Extract corresponding vectors from the pretrained word embedding

Next, we define a function to generate vectors for a collection of reviews. It is essentially a simpler form of the function for pretrained word embeddings – it is simpler as we do not need to worry about out-of-vocabulary words.

In [23]:
def assemble_embedding_vectors(data):
  out = None
  for item in data:    # Loop through every IMDB review
    vec = s2v_model.embed_sentence(" ".join(str(i) for i in item if i))     # Extract embedding vectors for every word in review, now we dont need to handle out-of-vocab words
    if vec is not None:                            # Edge case handling
      if out is not None:
        out = np.concatenate((out, vec), axis=0)    # Concatenate row vector to output Numpy array
      else:
        out = vec
    else:
      pass

  return out

### Preparing and assembling Dataset

In [24]:
# shuffle raw data first
def unison_shuffle_data(data, header):
    p = np.random.permutation(len(header))
    data = data[p]
    header = np.asarray(header)[p]
    return data, header

# load data in appropriate form
def load_data(path):
  data, sentiments = [], []
  for folder, sentiment in (("neg", 0), ("pos", 1)):
    folder = os.path.join(path, folder)
    for name in os.listdir(folder):    # Go through every file in current folder
      with open(os.path.join(folder, name), "r") as reader:
        text = reader.read()
      # Apply tokenization, stopword analysis routines
      text = tokenize(text)
      text = stop_word_removal(text)
      text = reg_expressions(text)
      # Track corresponding text and sentiment labels
      data.append(text)
      sentiments.append(sentiment)
  # Convert to Numpy array
  #print(data)
  data_np = np.array(data)
  #print(data_np[:10])
  data, sentiments = unison_shuffle_data(data_np, sentiments)

  return data, sentiments

In [25]:
train_path = os.path.join("aclImdb", "train")
test_path = os.path.join("aclImdb", "test")
raw_data, raw_header = load_data(train_path)

print(raw_data.shape)
print(len(raw_header))

(25000,)
25000


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [26]:
# Subsample required number of samples
random_indices = np.random.choice(range(len(raw_header)), size=(n_sample * 2,), replace=False)
data_train = raw_data[random_indices]
header = raw_header[random_indices]

del raw_data, raw_header # huge and no longer needed, get rid of it

print("DEBUG::data_train::")
print(data_train[:10])

DEBUG::data_train::
[list(['i', 'know', 'i', 'missed', 'here', 'i', 'cant', 'believe', 'positive', 'comments', 'many', 'people', 'film', 'i', 'thought', 'silly', 'bit', 'top', 'i', 'like', 'performances', 'gregg', 'henry', 'michael', 'rooker', 'however', 'others', 'just', 'boringbr', 'br', 'now', 'i', 'like', 'b', 'movies', 'i', 'really', 'do', 'bit', 'alphabet', 'me', 'i', 'saw', 'someone', 'compare', 'humor', 'horror', 'army', 'of', 'darkness', 'shaun', 'dead', 'well', 'on', 'par', 'the', 'reanimator', 'you', 'must', 'joking', 'i', 'find', 'film', 'funny', 'tried', 'make', 'effort', 'possibly', 'much', 'effort', 'failed', 'opinion', 'by', 'time', 'i', 'hit', 'rd', 'th', 'oneliner', 'i', 'rolling', 'eyes', 'checking', 'watchbr', 'br', 'there', 'definitely', 'homages', 'made', 'several', 'films', 'always', 'cool', 'kind', 'like', 'inside', 'joke', 'us', 'horror', 'fans', 'but', 'may', 'lack', 'original', 'thought', 'admittedly', 'nice', 'special', 'effects', 'good', 'gore', 'cant', 'ca

Display sentiments and their frequencies in the dataset, to ensure it is roughly balanced between classes.

In [27]:
unique_elements, counts_elements = np.unique(header, return_counts=True)

print("Sentiments and their frequencies:")
print(unique_elements)
print(counts_elements)

Sentiments and their frequencies:
[0 1]
[ 992 1008]


We can now use this function to extract sent2vec embedding vectors for each review.

In [28]:
embedding_vectors = assemble_embedding_vectors(data_train) 
print(embedding_vectors)

[[ 0.04913538 -0.01056214 -0.04326244 ...  0.09432245 -0.16622916
   0.18938614]
 [ 0.23617683 -0.07827432  0.012788   ... -0.23668842 -0.07756373
   0.10961107]
 [ 0.0593676  -0.09905271 -0.00589911 ... -0.09976954  0.13383257
   0.19327204]
 ...
 [ 0.01137837  0.03909565 -0.11264261 ... -0.02143067 -0.0557724
   0.21194567]
 [ 0.15456685 -0.14718585  0.01531324 ...  0.02665597 -0.02107718
   0.18258451]
 [-0.07354186  0.08375151 -0.10282756 ...  0.02766571 -0.03726078
   0.27708924]]


These can now be used as feature vectors for the same logistic regression and random forest.

As the very last step of preparing the sentiment dataset for training by our baseline classifiers, we split it into independent training and testing or validation sets. This will allow us to evaluate the performance of the classifier on a set of data that was not used for training, an important thing
to ensure in machine learning practice. We elect to use 70% of the data for training, and 30% for testing/validation afterwards.

In [29]:
data = embedding_vectors
del embedding_vectors

idx = int(0.7 * data.shape[0])


# 70% of data for training
train_x = data[:idx, :]
train_y = np.array(header[:idx])

# remaining 30% for testing
test_x = data[idx:, :]
test_y = np.array(header[idx:])

print("train_x/train_y list details, to make sure it is of the right form:")
print(len(train_x))
print(train_x[:5])
print(len(train_y))
print(train_y[:5])

train_x/train_y list details, to make sure it is of the right form:
1400
[[ 0.04913538 -0.01056214 -0.04326244 ...  0.09432245 -0.16622916
   0.18938614]
 [ 0.23617683 -0.07827432  0.012788   ... -0.23668842 -0.07756373
   0.10961107]
 [ 0.0593676  -0.09905271 -0.00589911 ... -0.09976954  0.13383257
   0.19327204]
 [-0.02144677 -0.10993782 -0.11229798 ...  0.11676594 -0.10136148
   0.12036694]
 [-0.09403957 -0.12461621  0.00794382 ...  0.04140427 -0.07364064
  -0.0288259 ]]
1400
[0 0 0 0 0]


## Domain Adaptation

The idea of domain adaptation, one of the oldest and mostprominent
ideas in transfer learning. An implicit assumption is made often by machine learning practitioners – that the data during the inference phase will come from the same distribution as the data that was used for training. This is of course rarely true in practice.

Enter domain adaptation to attempt to address this issue. Let’s define domain as a particular distribution of data for a specific task. Assume that we are given a source domain and an algorithm that has been trained to perform well on data in that domain. The goal of domain adaptation is to modify, or adapt, data in a different target domain in such a way that the pretrained knowledge from the source domain can be applicable to faster learning and/or direct
inference in the target domain. 

A variety of approaches ranging from multi-task learning - where learning on different data distributions occurs simultaneously, to coordinate transformations that enable more effective learning on a single combined feature space, and methods that exploit measures of similarity between the source and target domains to select which data should be used for training, have been explored.

We apply a simple autoencoding approach to “project” samples in the target domain into the source domain feature space. An autoencoder is a system that can learn to reconstruct inputs with very high accuracy, typically by encoding them into an efficient latent representation and learning to decode the said representation. 

A technical way of describing the process of reconstructing input is “learning the identity function”. They have traditionally been heavily used
in model dimensionality reduction applications, since the latent representation is often of smaller dimension than the original space from which the encoding happens, and the said dimension value can also be picked to strike the right balance between computational efficiency and
accuracy14. In the extreme favorable scenario, improvements can be obtained with no labeled data in the target domain. This is typically referred to as **zero-shot domain adaptation**.

The idea of zero-short transfer learning arises in many contexts. You can think of it as a sort of “holy grail” of transfer learning, since obtaining labeled data in the target domain can be an expensive exercise. 

Here, we will explore whether a classifier trained to predict the polarity of
IMDB movie reviews can be used to predict the polarity of reviews in some other domain. For example, can a classifier trained on IMDB review data predict polarity of book reviews or DVD reviews which are obtained from a completely different data source?

## Train Shallow Model for IMDB Reviews

Let's train classification model on IMDB movie review dataset.


In [30]:
input_shape = (len(train_x[0]), )

# Input must match the dimension of the sent2vec vectors
sent2vec_vectors = Input(shape=input_shape)
# Dense neural layer trained on top of the sent2vec vectors
dense = Dense(512, activation="relu")(sent2vec_vectors)
# Apply dropout to reduce overfitting
dense = Dropout(0.3)(dense)

# Output indicates a single binary classifier - is review “positive” or “negative”?
output = Dense(1, activation="sigmoid")(dense)

model = Model(inputs=sent2vec_vectors, outputs=output)

In [31]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
history = model.fit(train_x, train_y, validation_data=(test_x, test_y), batch_size=32, epochs=10, shuffle=True)

Epoch 1/10
44/44 [==============================] - 4s 7ms/step - loss: 0.6277 - accuracy: 0.6713 - val_loss: 0.4511 - val_accuracy: 0.7917
Epoch 2/10
44/44 [==============================] - 0s 4ms/step - loss: 0.4310 - accuracy: 0.8155 - val_loss: 0.3934 - val_accuracy: 0.8200
Epoch 3/10
44/44 [==============================] - 0s 4ms/step - loss: 0.3725 - accuracy: 0.8376 - val_loss: 0.3857 - val_accuracy: 0.8333
Epoch 4/10
44/44 [==============================] - 0s 4ms/step - loss: 0.3202 - accuracy: 0.8766 - val_loss: 0.4140 - val_accuracy: 0.8133
Epoch 5/10
44/44 [==============================] - 0s 4ms/step - loss: 0.3123 - accuracy: 0.8713 - val_loss: 0.3820 - val_accuracy: 0.8300
Epoch 6/10
44/44 [==============================] - 0s 4ms/step - loss: 0.2857 - accuracy: 0.8816 - val_loss: 0.3921 - val_accuracy: 0.8267
Epoch 7/10
44/44 [==============================] - 0s 4ms/step - loss: 0.2537 - accuracy: 0.9025 - val_loss: 0.3975 - val_accuracy: 0.8250
Epoch 8/10
44/44 [==

We found that the performance of this classifier was about 81% at the hyperparameter values specified.


## Test Trained Model on Book Reviews from MDSD

A rich repository of reviews is available. Indeed, one of the most
prominently and heavily explored datasets in the space of NLP domain adaptation happens to be a collection of reviews for different product categories on Amazon - The [Multi-Domain Sentiment Dataset](https://www.cs.jhu.edu/~mdredze/datasets/sentiment/). Many categories are represented within this dataset - a total of 25. Within this dataset, we picked the product category of book reviews, feeling that this was sufficiently different from IMDB reviews to present a challenging test-case.

The data in this dataset is contained in a markup language format, where tags are used to define various elements, and is organized by category and polarity into separate files. It suffices to note for our purposes that reviews are contained within appropriately named
`<review_text>...</review_text>` tags. 

Given this information, the code can be used to load positive and negative book reviews and prepare them for analysis.

In [32]:
def parse_MDSD(data):
  out_lst = []
  for i in range(len(data)):
      txt = ""
      # Locate the first line of review, combine all subsequent characters until end tag into review text
      if (data[i]=="<review_text>\n"):
          j=i
          while(data[j]!="</review_text>\n"):
              txt = txt+data[j]
              j = j+1
          text = tokenize(txt)
          text = stop_word_removal(text)
          text = reg_expressions(text)
          out_lst.append(text)
          
          #print(txt)
          #print(text)
          
  return out_lst

In [33]:
# Read lines from source text file both positive and negative reviews by leveraging defined function
with open("books.negative.review", "r", encoding="latin1") as my_file:
  data = my_file.readlines()
neg_books = parse_MDSD(data)
len(neg_books)

1000

In [34]:
with open("books.positive.review", "r", encoding="latin1") as my_file:
  data = my_file.readlines()
pos_books = parse_MDSD(data)
len(pos_books)

1000

In [35]:
# Create labels for the positive and negative classes
header = [0]*len(neg_books)
header.extend([1]*len(pos_books))

# append, shuffle and extract corresponding sent2vec vectors
neg_books.extend(pos_books)
MDSD_data = np.array(neg_books)

data, sentiments = unison_shuffle_data(np.array(MDSD_data), header)

len(sentiments)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  import sys


2000

Having loaded the book review text and prepared it for further processing, we now test the trained IMDB classifier from the previous section directly on the target data to see how accurate it is without any processing.

Let's try using the IMDB classifier directly on book review data...

In [36]:
embedding_vectors = assemble_embedding_vectors(data)
print(embedding_vectors)
sentiments = np.asarray(sentiments)

[[-0.10247996  0.05756732  0.24177921 ... -0.03724968 -0.2285876
   0.09678674]
 [-0.00950656 -0.12373678 -0.08567187 ...  0.17987318 -0.16593893
   0.15151091]
 [-0.12742344 -0.19568436 -0.1897929  ... -0.10396274 -0.13525589
   0.22625637]
 ...
 [-0.03482994 -0.13481432  0.01015367 ...  0.03452443 -0.30464894
   0.24166287]
 [-0.15266658 -0.10943983 -0.16178188 ... -0.00949576 -0.12323287
   0.07225277]
 [-0.15517686 -0.4002986   0.24433103 ... -0.02800336 -0.19798745
   0.18141942]]


In [37]:
# evaluate IMDB classifier on books directly
print(model.evaluate(x=embedding_vectors, y=sentiments))
print(model.metrics_names)

63/63 [==============================] - 0s 2ms/step - loss: 0.6135 - accuracy: 0.7510
[0.6135140657424927, 0.7509999871253967]
['loss', 'accuracy']


## Adaptation of Book Review Domain via Autoencoder

We train an autoencoder to reconstruct the IMDB data. The autoencoder takes the form of a shallow neural network that is similar to the multi-task layers.

A major difference from the previous neural networks is that since this is a regression problem, there is no activation in the output layer. The encoding dimension `encoding_dim` was empirically tuned for the right balance of accuracy and computing cost.

In [38]:
encoding_dim = 30

# Input size must be the same as the dimension of the sent2vec vectors
input_shape = (len(train_x[0]), )

sent2vec_vectors = Input(shape=input_shape)

# Encode into a space of specified latent dimension encoding_dim
encoder = Dense(encoding_dim, activation="relu")(sent2vec_vectors)
drop_out = Dropout(0.3)(encoder)

# Decode from space of specified latent dimension back into the sent2vec space
decoder = Dense(encoding_dim, activation="relu")(drop_out)
drop_out = Dropout(0.3)(decoder)

output = Dense(len(train_x[0]))(drop_out)

autoencoder = Model(inputs=sent2vec_vectors, outputs=output)

We train the autoencoder for 50 epochs, which takes only a few seconds, by setting both the input and output to the IMDB sent2vec vectors.

In [40]:
# Specify two loss functions (both binary_crossentropy in our case)
autoencoder.compile(optimizer="adam", loss="mse", metrics=["mse", "mae"])
# Specify training and validation data for each input
autoencoder.fit(train_x, train_x, validation_data=(test_x, test_x), batch_size=32, epochs=50, shuffle=True)

Epoch 1/50
44/44 [==============================] - 1s 7ms/step - loss: 0.0084 - mse: 0.0084 - mae: 0.0718 - val_loss: 0.0080 - val_mse: 0.0080 - val_mae: 0.0702
Epoch 2/50
44/44 [==============================] - 0s 4ms/step - loss: 0.0084 - mse: 0.0084 - mae: 0.0718 - val_loss: 0.0080 - val_mse: 0.0080 - val_mae: 0.0701
Epoch 3/50
44/44 [==============================] - 0s 4ms/step - loss: 0.0081 - mse: 0.0081 - mae: 0.0708 - val_loss: 0.0080 - val_mse: 0.0080 - val_mae: 0.0700
Epoch 4/50
44/44 [==============================] - 0s 4ms/step - loss: 0.0082 - mse: 0.0082 - mae: 0.0709 - val_loss: 0.0080 - val_mse: 0.0080 - val_mae: 0.0700
Epoch 5/50
44/44 [==============================] - 0s 4ms/step - loss: 0.0083 - mse: 0.0083 - mae: 0.0713 - val_loss: 0.0080 - val_mse: 0.0080 - val_mae: 0.0699
Epoch 6/50
44/44 [==============================] - 0s 5ms/step - loss: 0.0081 - mse: 0.0081 - mae: 0.0708 - val_loss: 0.0079 - val_mse: 0.0079 - val_mae: 0.0698
Epoch 7/50
44/44 [==========

We use mean squared error (mse) as loss function for this regression problem, and mean absolute error (mae) as an additional metric. A minimum validation mae value of approximately 0.06 was achieved.

We next project the book reviews into the IMDB feature space using the autoencoder trained to reconstruct such features above. This just means we preprocess the book review feature vectors using the autoencoder. We then repeat the accuracy evaluation experiment of the IMDB classifier on these preprocessed vectors as input:

In [41]:
# transform EmbeddingVectors and sentiments with autoencoder.predict and then evaluate IDMB model again
embedding_vectors_projected = autoencoder.predict(embedding_vectors)
print(embedding_vectors_projected.shape)

print(model.evaluate(x=embedding_vectors_projected,y=sentiments))
print(model.metrics_names)

(2000, 600)
63/63 [==============================] - 0s 2ms/step - loss: 0.7478 - accuracy: 0.5945
[0.747765302658081, 0.5945000052452087]
['loss', 'accuracy']


## Conclusions

An accuracy of approximately 74% was now observed, demonstrating an improvement about 1% and an instance of zero-shot domain adaptation. Repeating this several times, we found the improvement to consistently stay around 0.5-1%, giving us confidence that the autoencoding domain adaptation did indeed lead to some positive transfer.